# Controller - Sensor Mapping Isolation

## Imports

In [ ]:
import sys
import time
import matplotlib.pyplot as plt
import cv2
import math

import delivery_driver as dd

sys.path.append("../../deliveryrobot")
from utilities.utilities import *
from sensors.camera.apriltagsensor import *
from sensors.calibration.camera_calibration import *
from navigation.slam.onlineslam import *
from navigation.astar.astar import *
from kinematics.movementai import *

## Robot Initialization

In [ ]:
# Robot Instance initialization
start_time = time.time()

robot = dd.DeliveryRobot()

print(f"Robot initialization complete in {time.time()-start_time} seconds")

# Apriltag Sensor initialization
start_time = time.time()

sensor = AprilTagSensor(cal_dir)

print(f"Sensor calibration and initialization in {time.time()-start_time} seconds")

april_live = "live_april"

# SLAM Instance initialization
start_time = time.time()

slam = OnlineSLAM(3)

print(f"SLAM initialization in {time.time()-start_time} seconds")

# Astar Instance initialization
start_time = time.time()

scalar = 1
astar = Astar(
    beam_resolution=10,
    beam_range=np.pi/4,
    max_distance=0.2*scalar,
    heuristic_weight=1000/scalar,
    cost=1,
    fos=1,
    robot_radius_m=0.015*scalar
)
astar.debug = False
astar.verbose = False

print(f"Astar initialization in {time.time()-start_time} seconds")

In [ ]:
# sensor measurement
start_time = time.time()

april_img = robot.take_picture(image_dir + "/testing/pose_test/")
measurements={}
obstacles = {}
sensor.detect(april_img, measurements)

print(f"Sensor detections in {time.time()-start_time} seconds")
print(measurements)

# SLAM measurement
start_time = time.time()

slam.set_world_frame(int(list(measurements.keys()).index("1")))
slam.process_measurements(measurements)
slam.map_update(False)

print(f"SLAM estimation in {time.time()-start_time} seconds")
print(slam.get_map())

In [ ]:
"""import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)"""

In [ ]:
"""import traitlets

print(controller.axes)

left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.robot.left_motor, 'value'), transform=lambda x: 0.2 * -x)
right_link = traitlets.dlink((controller.axes[1], 'value'), (robot.robot.right_motor, 'value'), transform=lambda x: 0.2 * -x)"""

In [ ]:
bars = []
x_sensor = []
y_sensor = []
psi_sensor = []
x_slam = []
y_slam = []
psi_slam = []
measurements_last = {}
n = -1
t = 0

goal_id = "1"

robot.robot.set_motors(0.2,0.2)
start_time = time.time()

# Function that runs the loop
while t < 20:
    
    print(f"------------------ Timestamp {t} ------------------")
    # AprilSensor
    april_img = robot.take_picture(image_dir + "/testing/pose_test/")
    measurements = {}
    sensor.detect(april_img, measurements)

    print()
    print(april_img)
    print(f"Sensor Measurements: {measurements}")
    print()

    # SLAM
    if measurements and goal_id in measurements:
        slam.process_measurements(measurements)
        slam.process_movement(0.1, 0.0267)
    else:
        print("WARNING: NO SENSOR MEASUREMENTS")
        slam.process_movement(0.1, 0.0267, meas=False)
    
    
    slam.map_update(motion=True)
        
    print()
    print(slam.get_map())
    print()

    # AStar
    print(slam.get_map())
    robot_state = slam.get_map()["ROBOT"]
    goal_state = slam.get_map()[goal_id]
    astar_test = astar.astar_move(robot_state, obstacles, goal_state)

    print()
    astar_test.print_info()
    print()

    # update arrays
    if measurements and goal_id in measurements:
        x_sensor_t, y_sensor_t, psi_sensor_t = measurements[goal_id]
        bars.append(0)
    else:
        bars.append(1)
    x_slam_t, y_slam_t, psi_slam_t = slam.get_map()["ROBOT"]
    x_sensor.append(-x_sensor_t)
    y_sensor.append(-y_sensor_t)
    psi_sensor.append(-psi_sensor_t)
    x_slam.append(x_slam_t)
    y_slam.append(y_slam_t)
    psi_slam.append(psi_slam_t)

    
    t += 1
    
robot.robot.stop()
robot.step_backward(0.4,(time.time() - start_time)/2.5)

robot.robot.stop()

In [ ]:
robot.robot.stop()

In [ ]:
import matplotlib.pyplot as plt


def plot_estimations(x_sensor=None, y_sensor=None, psi_sensor=None,
                     x_slam=None, y_slam=None, psi_slam=None, time=None, bars=None):
    """
    Plots the x, y, and psi (orientation) estimations for sensor data and SLAM,
    with optional vertical bars. If any of the arrays are empty or None, that data won't be plotted.

    Parameters:
    x_sensor (list or np.ndarray): X positions from sensors.
    y_sensor (list or np.ndarray): Y positions from sensors.
    psi_sensor (list or np.ndarray): Psi (orientation) from sensors.
    x_slam (list or np.ndarray): X positions from SLAM.
    y_slam (list or np.ndarray): Y positions from SLAM.
    psi_slam (list or np.ndarray): Psi (orientation) from SLAM.
    time (list or np.ndarray, optional): Time or sample indices. Defaults to indices of the arrays.
    bars (list or np.ndarray, optional): Binary array where 1s indicate positions to add vertical bars.

    Returns:
    None: The function directly shows the plots.
    """
    # Set defaults for time and bars if not provided
    if time is None:
        time = range(len(x_slam)) if x_slam else range(len(x_sensor))  # Use index range based on available data
    
    if bars is None:
        bars = [0] * len(time)  # Default to no bars if not provided

    # Create the subplots
    fig, axs = plt.subplots(3, 1, figsize=(10, 12))

    # Plot x positions
    if x_sensor:
        axs[0].plot(time, x_sensor, label='Sensor Data', marker='o')
    if x_slam:
        axs[0].plot(time, x_slam, label='SLAM', marker='x')
    axs[0].set_title('X Position Comparison')
    axs[0].set_xlabel('Time')
    axs[0].set_ylabel('X Position')
    axs[0].legend()

    # Plot y positions
    if y_sensor:
        axs[1].plot(time, y_sensor, label='Sensor Data', marker='o')
    if y_slam:
        axs[1].plot(time, y_slam, label='SLAM', marker='x')
    axs[1].set_title('Y Position Comparison')
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Y Position')
    axs[1].legend()

    # Plot psi (orientation)
    if psi_sensor:
        axs[2].plot(time, psi_sensor, label='Sensor Data', marker='o')
    if psi_slam:
        axs[2].plot(time, psi_slam, label='SLAM', marker='x')
    axs[2].set_title('Psi (Orientation) Comparison')
    axs[2].set_xlabel('Time')
    axs[2].set_ylabel('Psi (Orientation)')
    axs[2].legend()

    # Add vertical bars if any
    for i, t in enumerate(time):
        if bars[i] == 1:
            axs[0].axvline(x=t, color='red', linestyle='--', linewidth=1)
            axs[1].axvline(x=t, color='red', linestyle='--', linewidth=1)
            axs[2].axvline(x=t, color='red', linestyle='--', linewidth=1)

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()
    

# Example usage where x_slam and y_slam are provided but others are empty
plot_estimations(x_sensor, y_sensor, psi_sensor,
                     x_slam, y_slam, psi_slam, bars=bars)

In [ ]:
print(psi_slam)

In [ ]:
print(psi_sensor)